# Adaptive Evaluations with Scorebook - Evaluating an OpenAI GPT Model

This quick-start guide showcases an adaptive evaluation of OpenAI's GPT-4o Mini model.

We recommend that you first see our [getting started quick-start guide](https://colab.research.google.com/github/trismik/scorebook/blob/main/tutorials/quickstarts/getting_started.ipynb) if you have not done so already, for more of a detailed overview on adaptive testing and setting up Trismik credentials.

## Prerequisites

- **Trismik API key**: Generate a Trismik API key from the [Trismik dashboard's settings page](https://app.trismik.com/settings).
- **Trismik Project Id**: We recommend you use the project id generated in the [Getting Started Quick-Start Guide](https://colab.research.google.com/github/trismik/scorebook/blob/main/tutorials/quickstarts/getting_started.ipynb).
- **OpenAI API key**: Generate an OpenAI API key from [OpenAI's API Platform](https://openai.com/api/).

## Install Scorebook

In [ ]:
!pip install scorebook
# if you're running this locally, please run !pip install scorebook"[examples, providers]"


## Setup Credentials

Enter your Trismik API key, project id and OpenAI API Key below.

In [ ]:
# Set your credentials here
TRISMIK_API_KEY = "your-trismik-api-key-here"
TRISMIK_PROJECT_ID = "your-trismik-project-id-here"
OPENAI_API_KEY = "your-openai-api-key-here"

## Login with Trismik API Key

In [ ]:
from scorebook import login

# Login to Trismik
login(TRISMIK_API_KEY)
print("✓ Logged in to Trismik")

## Define an Inference Function

To evaluate a model with Scorebook, it must be encapsulated within an inference function. An inference function must accept a list of model inputs, pass these to the model for inference, collect and return outputs generated.

An inference function can be defined to encapsulate any model, local or cloud-hosted. There is flexibility in how an inference function can be defined, the only requirements are the function signature. An inference function must,

Accept:

- A list of model inputs.
- Hyperparameters which can be optionally accessed via kwargs.

Return

- A list of parsed model outputs for scoring.

In [ ]:
from openai import OpenAI
from typing import Any, List

client = OpenAI(api_key=OPENAI_API_KEY)

# define an inference function for GPT-4o Mini.
def gpt4o_mini(inputs: List[Any], **hyperparameters: Any) -> List[Any]:
    """Process inputs through OpenAI's API. Handles both multiple-choice and open-ended items."""

    outputs = []
    for idx, input_item in enumerate(inputs):

        choices = input_item.get("choices")

        if choices:
            # Multiple-choice: format with choices, ask for letter
            prompt = (
                str(input_item.get("question", ""))
                + "\nOptions:\n"
                + "\n".join(
                    f"{choice['id']}: {choice['text']}"
                    for choice in choices
                )
            )
            system_msg = hyperparameters.get(
                "system_message",
                "Answer with only the letter of the correct option."
            )
        else:
            # Open-ended: just the question, ask for free text
            prompt = str(input_item.get("question", ""))
            system_msg = hyperparameters.get(
                "system_message_open_ended",
                "Answer the question. Place your final answer between <answer> and </answer> tags."
            )

        # Build messages for OpenAI API
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt},
        ]

        # Call OpenAI API and extract output from the response
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                temperature=0.7,
            )
            output = response.choices[0].message.content.strip()

            # For open-ended: extract from <answer> tags if present
            if not choices and "<answer>" in output:
                start = output.rfind("<answer>") + len("<answer>")
                end = output.rfind("</answer>")
                if end > start:
                    output = output[start:end].strip()

        except Exception as e:
            output = f"Error: {str(e)}"

        outputs.append(output)

    return outputs

## Run an Adaptive Evaluation

When running an adaptive evaluation, we can use any single or multiple adaptive datasets and specify a split to be evaluated.

### Multiple-Choice Adaptive Evaluation

In [ ]:
from scorebook import evaluate

# Run adaptive evaluation
results = evaluate(
    inference = gpt4o_mini,
    datasets = "trismik/figQA:adaptive",
    hyperparameters = {"system_message": "Answer the question with only the letter of the correct option. No additional text or context"},
    split = "validation",
    experiment_id = "GPT-4o-Mini-Adaptive-Evaluation",
    project_id = TRISMIK_PROJECT_ID,
)

# Print the adaptive evaluation results
print("✓ Adaptive evaluation complete!")
print("Results: ", results[0]["score"])

### Open-Ended Adaptive Evaluation

Scorebook also supports open-ended adaptive evaluations where the model provides free-text answers instead of selecting from multiple choices. The same inference function handles both item types by checking for the presence of `choices` in the input item.

In [ ]:
# Run open-ended adaptive evaluation
results_open_ended = evaluate(
    inference=gpt4o_mini,
    datasets="fingpt_convfinqa_test:adaptive",
    hyperparameters={
        "system_message": "Answer with only the letter of the correct option.",
        "system_message_open_ended": "Answer the question concisely. Place your final answer between <answer> and </answer> tags.",
    },
    split="validation",
    experiment_id="GPT-4o-Mini-Open-Ended-Adaptive",
    project_id=TRISMIK_PROJECT_ID,
)

# Print the open-ended adaptive evaluation results
print("✓ Open-ended adaptive evaluation complete!")
print("Results: ", results_open_ended[0]["score"])

---

## Next Steps

- [Adaptive Testing White Paper](https://docs.trismik.com/adaptiveTesting/adaptive-testing-introduction/): An in depth overview of the science behind the adaptive testing methodology.
- [Dataset Page](https://dashboard.trismik.com/datasets): Trismik's full set of currently adaptive datasets from the Trismik dashboard.
- [Scorebook Docs](https://docs.trismik.com/scorebook/introduction-to-scorebook/): Scorebook's full documentation.
- [Scorebook Repository](https://github.com/trismik/scorebook): Scorebook is an open-source library, view the code and more examples.